# Introduction: Using Trained Model

The purpose of this noteobook is to use the trained word level model in order to make predictions. We can look at using both the model trained with pre-trained embeddings and the model with embeddings that were trained from scratch.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import HTML

InteractiveShell.ast_node_interactivity = 'all'

In [3]:
from keras import Model
from keras.models import load_model
import warnings
warnings.filterwarnings('ignore', category = RuntimeWarning)
warnings.filterwarnings('ignore', category = UserWarning)

BATCH_SIZE = 2048
RANDOM_STATE = 50

import numpy as np
import pandas as pd
from utils import get_model, find_closest, get_sequences, create_train_valid,  generate_output, guess_human

Using TensorFlow backend.


# Model Trained with Own Embeddings

In [4]:
model, embeddings, word_idx, idx_word = get_model('train-embeddings-rnn')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 100)         1619200   
_________________________________________________________________
lstm_2 (LSTM)                (None, 64)                42240     
_________________________________________________________________
dense_3 (Dense)              (None, 128)               8320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 16192)             2088768   
Total params: 3,758,528
Trainable params: 3,758,528
Non-trainable params: 0
_________________________________________________________________


## Inspect Trained Embeddings

In [5]:
find_closest('the', embeddings, word_idx, idx_word)

Query: the

Word: the             Cosine Similarity: 1.0
Word: a               Cosine Similarity: 0.8156999945640564
Word: The             Cosine Similarity: 0.7764999866485596
Word: this            Cosine Similarity: 0.7674000263214111
Word: its             Cosine Similarity: 0.7583000063896179
Word: third           Cosine Similarity: 0.7434999942779541
Word: any             Cosine Similarity: 0.7361999750137329
Word: their           Cosine Similarity: 0.7300999760627747
Word: second          Cosine Similarity: 0.7290999889373779
Word: entire          Cosine Similarity: 0.7247999906539917


In [6]:
find_closest('a', embeddings, word_idx, idx_word)

Query: a

Word: a               Cosine Similarity: 1.0
Word: the             Cosine Similarity: 0.8156999945640564
Word: A               Cosine Similarity: 0.7390999794006348
Word: any             Cosine Similarity: 0.7271000146865845
Word: this            Cosine Similarity: 0.7246000170707703
Word: no              Cosine Similarity: 0.6948000192642212
Word: another         Cosine Similarity: 0.6919999718666077
Word: promising       Cosine Similarity: 0.6887999773025513
Word: its             Cosine Similarity: 0.6744999885559082
Word: unprecedented   Cosine Similarity: 0.6743000149726868


In [7]:
find_closest('.', embeddings, word_idx, idx_word)

Query: .

Word: .               Cosine Similarity: 1.0
Word: .”              Cosine Similarity: 0.755299985408783
Word: while           Cosine Similarity: 0.7282000184059143
Word: .&#8221         Cosine Similarity: 0.7235000133514404
Word: ##EQU2##        Cosine Similarity: 0.7148000001907349
Word: 10.             Cosine Similarity: 0.7027000188827515
Word: 74.             Cosine Similarity: 0.6988000273704529
Word: 7.              Cosine Similarity: 0.6833000183105469
Word: 18.             Cosine Similarity: 0.6807000041007996
Word: ?               Cosine Similarity: 0.675599992275238


In [8]:
find_closest('neural', embeddings, word_idx, idx_word)

Query: neural

Word: neural          Cosine Similarity: 1.0
Word: neuronal        Cosine Similarity: 0.6409000158309937
Word: Said            Cosine Similarity: 0.6399999856948853
Word: 3G              Cosine Similarity: 0.6276000142097473
Word: TCP/IP          Cosine Similarity: 0.6082000136375427
Word: Source          Cosine Similarity: 0.6029000282287598
Word: brain           Cosine Similarity: 0.566100001335144
Word: ARTMAP          Cosine Similarity: 0.5600000023841858
Word: search/sort     Cosine Similarity: 0.5591999888420105
Word: LTM             Cosine Similarity: 0.5540000200271606


In [9]:
find_closest('UNK', embeddings, word_idx, idx_word)

UNK has no pre-trained embedding.


# Evaluate on Test Data

In [19]:
data = pd.read_csv('../data/neural_network_patent_query.csv')
abstracts = list(data['patent_abstract'])
features, labels, sequences = get_sequences(abstracts, model_name = 'training-rnn')
X_train, X_test, y_train, y_test = create_train_valid(features, labels, num_words = len(word_idx))

There are 16192 unique words.
There are 318563 training sequences.


In [20]:
X_test.shape

(95569, 50)

In [21]:
y_test.shape

(95569, 16192)

In [22]:
print('Model metrics: log loss and accuracy')
model.evaluate(X_test, y_test, batch_size = 2048)

Model metrics: log loss and accuracy
95569/95569 [==============================] - 10s 108us/step


[4.737925765920241, 0.2671891513580688]

# Generate Output

In [14]:
seed_html, gen_html, a_html = generate_output(model, sequences, idx_word)
HTML(seed_html)
HTML(gen_html)
HTML(a_html)

In [15]:
seed_html, gen_html, a_html = generate_output(model, sequences, idx_word)
HTML(seed_html)
HTML(gen_html)
HTML(a_html)

# Output from Human or Machine?

In [16]:
def guess_human(model, sequences, training_length=50, new_words=50):
    """Produce 2 RNN sequences and play game to compare to actaul.
       Diversity is randomly set between 0.5 and 1.25"""
    
    diversity = np.random.uniform(0.5, 1.25)
    sequence, gen_list, actual = generate_output(model, sequences, training_length, 
                                                 diversity=diversity, return_output=True, n_gen = 2)
    gen_0, gen_1 = gen_list
    
    output = {'sequence': remove_spaces(' '.join(sequence)),
              'c0': remove_spaces(' '.join(gen_0)),
              'c1': remove_spaces(' '.join(gen_1)),
              'h': remove_spaces(' '.join(actual))}
    
    print(f"Seed Sequence: {output['sequence']}\n")
    
    choices = ['h', 'c0', 'c1']
          
    selected = []
    i = 0
    while len(selected) < 3:
        choice = random.choice(choices)
        selected.append(choice)
        print('\n')
        print(f'Option {i + 1} {output[choice]}')
        choices.remove(selected[-1])
        i += 1
    
    print('\n')
    guess = int(input('Enter option you think is human (1-3): ')) - 1
    print('\n')
    
    if guess == np.where(np.array(selected) == 'h')[0][0]:
        print('Correct')
        print('Correct Ordering', selected)
    else:
        print('Incorrect')
        print('Correct Ordering', selected)
          
    print('Diversity', round(diversity, 2))

In [17]:
training_length = 50

In [18]:
guess_human(model, sequences, idx_word)

NameError: name 'remove_spaces' is not defined

In [ ]:
guess_human(model, sequences, idx_word)

## Appendix: Injecting Diversity into Predictions

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [ ]:
def diversify(arr, diversity, plot = False):
    div = np.log(arr) / diversity
    exp_preds = np.exp(div)
    preds = exp_preds / np.sum(exp_preds)
    if plot:
        plt.figure(figsize = (10, 8));
        plt.subplot(2, 1, 1);
        sns.distplot(arr); plt.title('Original Distribution');
        plt.subplot(2, 1, 2);
        sns.distplot(preds); plt.title(f'Distribution with {diversity} diversity')
    probas = np.random.multinomial(1, preds, 1);
    return probas

In [ ]:
_ = diversify([0.1, 0.2, 0.4, 0.8, 0.9, 0.1, 0.3, 0.4], 0.5, plot = True)

In [ ]:
_ = diversify([0.1, 0.2, 0.4, 0.8, 0.9, 0.1, 0.3, 0.4], 2, plot = True)

In [ ]:
_ = diversify([0.1, 0.2, 0.4, 0.8, 0.9, 0.1, 0.3, 0.4], 10, plot = True)